<a href="https://colab.research.google.com/github/b02203054/Carvalue/blob/main/car_value_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
pip install git+git://github.com/scikit-learn/scikit-learn.git

  Cloning git://github.com/scikit-learn/scikit-learn.git to /tmp/pip-req-build-7xstque2
  Running command git clone -q git://github.com/scikit-learn/scikit-learn.git /tmp/pip-req-build-7xstque2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-learn: filename=scikit_learn-1.0.dev0-cp37-cp37m-linux_x86_64.whl size=18492276 sha256=24d7384afbed14fa245a019ac1d9fa97264c6144828970d2cbd98db0e7c3faed
  Stored in directory: /tmp/pip-ephem-wheel-cache-xfa2cz55/wheels/a1/50/0e/316ef2ff8d4cfade292bd20b49efda94727688a153382745a6
Successfully built scikit-learn
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
pip install sklearn

##  Car Price Prediction - Preprocessing

###  Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import re
import pprint
import joblib
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import sklearn

#Python iterators for efficient looping
import itertools

#Brute force parameter selection
from sklearn.model_selection import ParameterGrid

#pd.set_option('max_colwidth', 2000)
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

### Import Raw Data

In [ ]:
#Import data - austin, dallas, houston, and san antonio.  Two snapshots.

austin1=pd.read_json('/content/drive/My Drive/MIT5970/machine learning project - car value/austin_ford_2019-11-18.json')
dallas1=pd.read_json('/content/drive/My Drive/MIT5970/machine learning project - car value/dallas_ford_2019-11-18.json')
ep1=pd.read_json('/content/drive/My Drive/MIT5970/machine learning project - car value/ep_ford_2019-11-18.json')
houston1=pd.read_json('/content/drive/My Drive/MIT5970/machine learning project - car value/houston_ford_2019-11-18.json')
sa1=pd.read_json('/content/drive/My Drive/MIT5970/machine learning project - car value/sa_ford_2019_11_18.json')

austin2=pd.read_json('/content/drive/My Drive/MIT5970/machine learning project - car value/austin_ford_2019-12-29.json')
dallas2=pd.read_json('/content/drive/My Drive/MIT5970/machine learning project - car value/dallas_ford_2019-12-29.json')
ep2=pd.read_json('/content/drive/My Drive/MIT5970/machine learning project - car value/ep_ford_2019-12-29.json')
houston2=pd.read_json('/content/drive/My Drive/MIT5970/machine learning project - car value/houston_ford_2019-12-29.json')
sa2=pd.read_json('/content/drive/My Drive/MIT5970/machine learning project - car value/sa_ford_2019_11_18.json')

vehicle = pd.DataFrame()
for dframe in [austin1, dallas1, ep1, houston1, sa1, austin2, dallas2, ep2, houston2, sa2]:
    vehicle = vehicle.append(dframe, ignore_index = True)
display(f'vehicle has {len(vehicle)} rows:', vehicle.tail(1))

ValueError: ignored

In [ ]:
#The vehicle 'Body' is a comma-delimited list.  Join list into a single string.
vehicle['Body'] = vehicle['Body'].str.join(',')

#Remove whitespace.  Replace NaNs with None.
vehicle['SubLoc'] = vehicle['SubLoc'].str.strip()
SubLoc_dict={'SubLoc': {np.nan : 'None'}}
vehicle.replace(SubLoc_dict, inplace=True)

In [ ]:
#Add feature columns and view dataframe
print('vehicle length:', len(vehicle))
print('vehicle type:', type(vehicle))

vehicle.insert(2,'Location',np.nan)
vehicle.insert(3,'Year',np.nan)
vehicle.insert(4,'Year_in_Title',np.nan)
vehicle.insert(5,'Odometer',np.nan)
vehicle.insert(6,'RawMake',np.nan)
vehicle.insert(7,'Make',np.nan)
vehicle.insert(8,'Model',np.nan)
vehicle.insert(9,'Trim',np.nan)
vehicle.insert(10,'Seller',np.nan)
vehicle.insert(11,'Wreck',np.nan)
vehicle.insert(12,'Cab',np.nan)
vehicle.head(1)

In [ ]:
#Helper functions.

def match_regex_patt(df, target_col, regex_patt, no_match_value='None'):
    '''Returns regex_patt matches as list.  Case is ignored.'''
    matchList=[]
    cnt = 0
    for idx in df.index:
        m = re.search(regex_patt, df.loc[idx, target_col], flags=re.IGNORECASE) #re.search(pattern, string) gets first match
        if m is not None:
            matchList.append(m.group(1).lower())
        else:
            cnt+=1
            matchList.append(no_match_value)
    print(f'## In {target_col}, {cnt} missed matches for regex pattern: {regex_patt}.\n')
    return matchList

def findall_regex_patt(df, target_col, regex_patt, max_items, no_match_value='None'):
    '''Returns regex_patt matches found left-to-right as list.  Case is ignored.'''
    findallList=[]
    cnt = 0
    for idx in df.index:
        allm = re.findall(regex_patt, df.loc[idx, target_col] + " ", flags=re.IGNORECASE) #return all matches left-to-right       
        
        if len(allm) > 0:
            allm = allm[0:max_items]  #first two items
            #allm = sorted(allm)       #alphabetize 
            oneString = ("").join(allm).strip() #convert list to string and remove any spaces.
            findallList.append(oneString.lower())
        else:
            cnt+=1
            findallList.append(no_match_value)
    print(f'## In {target_col}, {cnt} missed matches for regex pattern: {regex_patt}.\n')
    return findallList

def findall_body_regex_patt(df, target_col, regex_patt, max_items, no_match_value='None'):
    '''Returns regex_patt matches found left-to-right as list.  Case is ignored.
       Each Model included in pattern.'''
    findallList=[]
    cnt = 0
    for idx in df.index:
        patt_w_model = df.loc[idx, "Model"] + regex_patt  #regex pattern is: Model + Pattern
        allm = re.findall(patt_w_model, df.loc[idx, target_col] + " ", flags=re.IGNORECASE) #return all matches left-to-right
        
        if len(allm) > 0:
            allm = allm[0:2]     #first two items
            #allm = sorted(allm)  #alphabetical 
            oneString = ("").join(allm).strip() #convert list to string and remove any spaces.
            findallList.append(oneString.lower())
        else:
            cnt+=1
            findallList.append(no_match_value)
    print(f'## In {target_col}, {cnt} missed matches for regex pattern: {regex_patt}.\n')
    return findallList


#****************************************Target Encoder******************************************

def target_encode_categ(encode_cols, train, test, target, encoder):
    '''Returns encoded categorical features.  Encoded feature is a blend of
    (1) ExpectedVal( trainTarget | FeatureClass) and 
    (2) "Prior": ExpectedVal(Target) over all training data.  
    
    Sets testTarget to NaN to stop data leakage.
    
    Encoder smoothing balances Class average vs Prior. Higher smoothing is stronger
    regularization.
    
    Arguments
    ---------
    train: training data including target Y
    test: test data including target Y
    target: target Y
    encoder: TargetEncoder(cols_to encode, smoothing_float_value).  
    
    See https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html
    for more parameters.'''
    
    trn=train.copy(); tst=test.copy();
    if target in tst.columns:   #Omit any test targets from encoding.  Prevents leakage.
        tst[target] = np.nan  
    fullX = trn.append(tst)
    fullY = fullX.pop(target)       
    fullX_enc = encoder.fit_transform(fullX, fullY)
    
    #Overwrite train/test features with encoded features.
    train[encode_cols] = fullX_enc.loc[train.index, encode_cols]
    test[encode_cols] = fullX_enc.loc[test.index, encode_cols]
    return train, test

def multivar_impute(train, test, target):
    '''Impute values with regression on numeric columns, excluding
    the target. Returns DataFrames.'''
    
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer
    num_cols_ex_targ = train.select_dtypes(exclude='object').columns.tolist()
    num_cols_ex_targ.remove(target)
    
    #Get imputed dfs.
    ii = IterativeImputer(max_iter=10, random_state=5)
    train_impX = pd.DataFrame(ii.fit_transform(train[num_cols_ex_targ]), index=train.index, columns=num_cols_ex_targ)
    test_impX = pd.DataFrame(ii.transform(test[num_cols_ex_targ]), index=test.index, columns=num_cols_ex_targ)

    #Overwrite original features with imputed features.
    train[num_cols_ex_targ] = train_impX
    test[num_cols_ex_targ] = test_impX
    return train, test

In [ ]:
#Get VehicleID from URL_Vehicle.    
vehIDList = match_regex_patt(df=vehicle, target_col='URL_Vehicle', regex_patt=r'/(\d+)\.html', no_match_value='None')    
vehicle['VehicleID'] = vehIDList

#Get vehicle Location from URL location.craigslist.org
vehLocList = match_regex_patt(df=vehicle, target_col='URL_Vehicle', regex_patt='https://(.+)\.craigslist', no_match_value='None')    
vehicle['Location'] = vehLocList

In [ ]:
pd.set_option('max_colwidth', 80)
vehicle.head(2)

In [ ]:
#Drop incomplete listing 5247.
vehicle.drop(index = 5247, inplace = True)

In [ ]:
#Build Year column from AttribDictionary.  Assert numeric format and none missing.
yrList=[]
for idx in vehicle.index:
    m = re.search(r'(\d+)', vehicle.loc[idx, 'AttribDictionary']['0'])  #re.search(pattern, string) gets first match
    yrList.append(m.group(1))  
vehicle['Year'] = yrList    
vehicle['Year'] = vehicle['Year'].astype('float').astype('int64')

#Keep Years 1995 and later.
print('Count of vehicles older than 1999 that were dropped:', sum(vehicle['Year'] < 1999))
filt = (vehicle['Year'] >= 1995)
vehicle = vehicle[filt]

assert vehicle['Year'].dtypes == 'int64',  "Year must be integer."
assert len(vehicle[vehicle['Year'].isna()]) == 0, "Year cannot have NAs."

In [ ]:
#EXTRACT Odometer & VIN from AttribDictionary.  
odomet=[]; vin = []; vinCNT=0;

for idx in vehicle.index:
    odo_success = 0
    vin_success = 0
    
    for k in vehicle.loc[idx, 'AttribDictionary'].keys(): #For each index, go through dict keys.
        od_m = re.search(r'<span>odometer: <b>(\d+)</b>', vehicle.loc[idx, 'AttribDictionary'][str(k)]) #re.search(pattern, string) gets first match
        vin_m = re.search(r'<span>VIN: <b>(\w+)</b>', vehicle.loc[idx, 'AttribDictionary'][str(k)]) #re.search(pattern, string) gets first match
                        
        if od_m is not None:
            odo_success = 1
            odomet.append(od_m.group(1))            
        if vin_m is not None:
            vin_success = 1
            vin.append(vin_m.group(1))          
            
    if odo_success != 1:
        odomet.append(np.nan)        
    if vin_success != 1:
        vinCNT += 1
        vin.append('None'+str(vinCNT))  #Assign unique value.  VIN is missing--not neccesarily duplicate listing.


vehicle['Odometer'] = odomet
vehicle['Odometer'] = vehicle['Odometer'].astype('float')
vehicle['VIN'] = vin

#SCRUB Odometer.
#Mine Odometer alternative from Body.
odomAlt=r'((\d+,?\.?\d+)(K\s| miles))'
odomAltList = match_regex_patt(df=vehicle, target_col='Body', regex_patt=odomAlt, no_match_value=np.nan)    
vehicle['RawOdomAlt'] = odomAltList

#Clean RawOdomAlt numbers.
vehicle['RawOdomAlt'] = vehicle['RawOdomAlt'].str.replace('[Kk]', '000').str.replace(',', '')\
                        .str.replace('.', '').str.replace('miles', '').str.strip()

#Set clean numbers less than 1,000 to np.nan.  Mileage this low is likely invalid.
vehicle['OdomAlt'] = vehicle['RawOdomAlt'].astype('float')
vehicle.loc[vehicle['OdomAlt'] < 1000, 'OdomAlt'] = np.nan

#Assign the OdomAlt value to NaN Odometer values.  Drop RawOdomAlt and OdomAlt.
vehicle.loc[vehicle['Odometer'].isna(), 'Odometer'] = vehicle.loc[vehicle['Odometer'].isna(), 'OdomAlt']
vehicle.drop(['RawOdomAlt', 'OdomAlt'], axis='columns', inplace=True, errors='igonre')

#Fix odometer typo.
vehicle.loc[9196, 'Odometer'] = vehicle.loc[9196, 'Odometer'] / 1000
vehicle.loc[19738, 'Odometer'] = vehicle.loc[19738, 'Odometer'] / 1000

### Seller, Make, Model, Price, CombMPG, Displacement

In [ ]:
#Create Seller feature using URL_vehicle.
vehicle['Seller']=vehicle['URL_Vehicle'].str.extract(r'(ct[o|d])')
vehicle['Seller']=vehicle['Seller'].str.replace('ctd', 'dealer')
vehicle['Seller']=vehicle['Seller'].str.replace('cto', 'owner')

In [ ]:
#Extract RawMake col from AttribDictionary in UPPERCASE.
rawMakeList=[]
for idx in vehicle.index:
    m = re.search(r'<b>(.+)</b>', vehicle.loc[idx, 'AttribDictionary']['0']) #re.search(pattern, string) gets first match
    if m is not None:
        rawMakeList.append(m.group(1))
    else:
        print('Missing Make/Model index:', idx)
        rawMakeList.append('None')
                 
vehicle['RawMake'] = rawMakeList

In [ ]:
vehicle.head(2)

In [ ]:
#Populate Make
make_regex=r'\s(ford)'
makeList = match_regex_patt(df=vehicle, target_col='RawMake', regex_patt=make_regex, no_match_value='None')    
vehicle['Make'] = makeList

#Populate Model.
model_regex=r'(bronco|buick|caravan|cargo|cmax|c-max|contour|crown|dually|e-150|e150|e-250|e250|e-350|e350|e-450|e450|ecoline|econoline|ecosport|edge|es350|escape|escort|excursion|expedition|explorer|f-150|f-250|f-350|f-450|f-550|f-650|f-750|f-800|f150|f250|f350|f450|f550|f650|f750|f800|fiesta|five|flex|focus|freestar|freestyle|fusion|lcf|lincoln|mustang|national|ranger|raptor|regency|scion|sport|starcraft|t-150|t-250|t-350|taurus|thunderbird|transit|transit-150|transit-250|transit-350|van|windstar)'
model_RawMake = match_regex_patt(df=vehicle, target_col='RawMake', regex_patt=model_regex, no_match_value='None')    
vehicle['Model'] = model_RawMake

#Populate Model.
model_Title = match_regex_patt(df=vehicle, target_col='Title', regex_patt=model_regex, no_match_value='None')
vehicle['model_Title'] = model_Title
vehicle.loc[vehicle['Model'] == 'None', 'Model'] = vehicle.loc[vehicle['Model'] == 'None', 'model_Title']

#Drop redundant column
vehicle.drop(['model_Title'], axis='columns', inplace=True)

In [ ]:
#Scrub Price - remove any $, periods, and spaces.  Note:  ORDER IS IMPORTANT!
#No raw string -- use "\" to escape next character.
vehicle['Price']=vehicle['Price'].str.replace('\,', '')
vehicle['Price']=vehicle['Price'].str.replace('\.00', '')
vehicle['Price']=vehicle['Price'].str.replace('\.0', '')
vehicle['Price']=vehicle['Price'].str.replace('\$', '')
vehicle['Price']=vehicle['Price'].str.replace('-', '')
vehicle['Price']=vehicle['Price'].str.replace(' ', '')
vehicle['Price']=vehicle['Price'].str.replace('\.', 'None')

In [ ]:
vehicle.head(2)

In [ ]:
#Insert CombMPG and displacement from modelYR.  
EPA = joblib.load('/content/drive/My Drive/MIT5970/machine learning project - car value/EPA.joblib')

#Backup vehicle indices before merge.
vehicle_idx_BAK = vehicle.index

#Merge data.  NOTE: Trim-level detail is unavailable in EPA data.  So ALL trim levels get the Model median.
#For example, a 2017 Mustang GT gets the same displacement as 2017 ecoboost.  More detail will improve the model.
vehicle = vehicle.merge(EPA, how='left', left_on=['Model','Year'], right_on=['clean_model','year'])

#Drop redundant columns.  Restore original vehicle indices that were reset by merge
vehicle.drop(columns=['clean_model', 'year'], inplace=True)
vehicle.rename(columns={'comb08' : 'CombMPG'}, inplace=True)
vehicle.index = vehicle_idx_BAK

In [ ]:
vehicle.head(1)

### Drop bad rows

In [ ]:
#Drop TRUCKMAX autos that are outside Texas.  
vehicle[vehicle['SubLoc'] == '(TRUCKMAX)']
drop_indices = vehicle[vehicle['SubLoc'] == '(TRUCKMAX)'].index
print(f'Drop {len(drop_indices)} TRUCKMAX listings outside Texas:', drop_indices)
vehicle.drop(index = drop_indices, inplace = True, errors = 'ignore')

#Make isn't 'ford'.  Drop corresponding rows.
filt = (vehicle['Make'] != 'ford')
drop_indices = vehicle[filt].index
print(f'Drop {len(drop_indices)} non-ford makes:', drop_indices)
vehicle.drop(index = drop_indices, inplace = True, errors = 'ignore')

In [ ]:
#Price is missing, None, or blank.  Drop corresponding rows.
filt = (vehicle['Price'].isna()) | (vehicle['Price'] == 'None') | (vehicle['Price'] == '')
drop_indices = vehicle[filt].index
print(f'## Drop {len(drop_indices)} missing price rows:', drop_indices, '\n')
vehicle.drop(index = drop_indices, inplace = True, errors = 'ignore')

#Price is "NoneNoneNone"
filt = (vehicle['Price'] == 'NoneNoneNone')
drop_indices = vehicle[filt].index
print(f'## Drop {len(drop_indices)} bad prices:', drop_indices)
vehicle.drop(index = drop_indices, inplace = True, errors = 'ignore')

#Cast Price as float.  Stop code execution if NaNs exist.
vehicle['Price'] = vehicle['Price'].astype('float')
assert len(vehicle[vehicle['Price'].isna()]) == 0, "Price cannot have NAs."

#Flag invalid "DOWN/DP" prices by searching Title/SubLoc.  Drop "DOWN" prices under $5,001.  
#Cannot be "DOWNTOWN".
downList=[]
for idx in vehicle.index:
    m = re.search(r'(DP |DO$|DOWN)[^T]', vehicle.loc[idx, 'Title'] + vehicle.loc[idx, 'Body']\
        + vehicle.loc[idx, 'SubLoc'] + " ", flags=re.IGNORECASE)
    if m is not None:
        downList.append('down')
    else:
        downList.append('None')
                 
vehicle['DownFlag'] = downList

filt = (vehicle['DownFlag'] == 'down') & (vehicle['Price'] < 5001)
drop_indices = vehicle[filt].index
print(f'## Drop {len(drop_indices)} DOWN prices under $5,001:', drop_indices)
vehicle.drop(index = drop_indices, inplace = True, errors = 'ignore')

#Drop very high or low prices.
filt2 = (vehicle['Price'] > 74999) | (vehicle['Price'] < 501)
drop_indices = vehicle[filt2].index
print(f'## Drop {len(drop_indices)} prices over $74,999 or under $501:', drop_indices)
vehicle.drop(index = drop_indices, inplace = True, errors = 'ignore')

In [ ]:
vehicle.head(2)

In [ ]:
#Fix Model names.
model_dict={
            'Model': {np.nan : 'None', 
                      '' : 'None',
                      'cmax' : 'c-max', 'e150' : 'e-150', 'e250' : 'e-250', 'e350' : 'e-350',
                      'f150' : 'f-150', 'f250' : 'f-250', 'f350' : 'f-350', 'f450' : 'f-450',
                     }
            }

vehicle.replace(model_dict, inplace=True)

In [ ]:
#Set Odometer=0 to NaN.
odZeroFilt = (vehicle['Odometer'] == 0)
zero_indices = vehicle[odZeroFilt].index
print(f'# {len(zero_indices)} Odometer values are zero.  Set to NaN.', zero_indices)
vehicle.loc[zero_indices, 'Odometer'] = np.nan

#Drop duplicate VINs.
print('\n# Duplicate VINs dropped:', vehicle.duplicated(subset=['VIN'], keep='first').sum())
vehicle.drop_duplicates(subset=['VIN'], inplace=True)

#Drop listings with the same Title/Odometer/Price.
print('\n# Duplicate Vehicles dropped:', vehicle.duplicated(subset=['Title', 'Odometer', 'Price'], keep='first').sum())
vehicle.drop_duplicates(subset=['Title', 'Odometer', 'Price'], inplace=True)

### Drop missing and commercial Models that lack EPA data.

In [ ]:
#Remove vehicle 'Model' is None.
filt = (vehicle['Model'] == 'None')
drop_indices = vehicle[filt].index
print(f'# Drop {len(drop_indices)} Models that are "None":', drop_indices)
vehicle = vehicle.drop(index = drop_indices, errors = 'ignore')

#Remove f-250, f-350, and f-450 -- no EPA data.  These vehicles are commercial, not personal.
filt = (vehicle['Model'] == 'f-250') | (vehicle['Model'] == 'f-350') | (vehicle['Model'] == 'f-450')
drop_indices = vehicle[filt].index
print(f'# Drop {len(drop_indices)} Models that are "f-250", "f-350", or "f-450":', drop_indices)
vehicle = vehicle.drop(index = drop_indices, errors = 'ignore')

## Drop Outliers

In [ ]:
# +++++++++++++++++ DROP OUTLIERS  ++++++++++++++++++++++++

# ---Train sqrt price residuals.---
# 13843 low price due to accident. Listing omits accident.
# Drop 3 sigma outliers

# ---Test sqrt price residuals.---
# No outliers dropped.  Some of the worst outliers are aftermarket-modified vehicles.

drop_indices=[13843,15627, 970, 149, 10850, 11624, 17701, 7413, 638, 4878, 1113, 9892, 4870, 2440, 12187, 40, 5843, 5236, 16436, 16433, 10731, 13365, 11575, 15677, 17267, 10975, 6216, 2814, 15390, 9556, 16664, 5109, 1668, 17335, 11946, 12902, 16443, 10962, 8413, 9739, 5722, 182, 969, 12196, 612, 11677, 7266, 10242, 789, 3319, 15182, 6734, 10700, 13711, 1991, 3192, 1210, 5812, 9751, 12317, 14674, 13518, 1303, 6097, 4102, 5797, 7488, 355, 16056, 12915, 7875, 5850, 15227, 15249, 3180, 16719, 13154]
print(f'Drop {len(drop_indices)} outliers:', drop_indices)
vehicle = vehicle.drop(index = drop_indices)

In [ ]:
#Catplot by Year and Price.  Year is effectively a left-skewed histogram.  Looks like Year^2 could be a good feature!
sns.set(font_scale=1.2)
c = sns.catplot(x='Year',y='Price', data=vehicle, height=8, aspect=1.2, s=6)  #s changes marker size
c.set_xticklabels(rotation=90)
plt.show()

In [ ]:
#Year histogram
y=sns.distplot(vehicle['Year'], bins=25, kde=True)
y.set_title("Normalized Year Histogram")
plt.show()

In [ ]:
vehicle

In [ ]:
vehicle = vehicle.drop(columns=['URL_Vehicle', 'Title', 'Year_in_Title', 'RawMake','Body','AttribDictionary', 'ImageDictionary','VehicleID','VIN'])

vehicle.dtypes



# For Decision tree 

In [ ]:
#basically import everything

import numpy as np 
import pandas as pd
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import train_test_split
import graphviz
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
import os

In [ ]:
# set up the variables and target
vehicle = vehicle.fillna(0)
decision_tree_df_x = vehicle[['Year','Odometer','Trim','Wreck','Cab','displ','CombMPG']].values

decision_tree_df_y = vehicle["Price"]


In [ ]:
# split the data into training group and test group
decision_tree_df_x_trainset, decision_tree_df_x_testset, decision_tree_df_y_trainset, decision_tree_df_y_testset = train_test_split(decision_tree_df_x, decision_tree_df_y, test_size=0.3, random_state=3)
print(np.shape(decision_tree_df_x_testset))

In [ ]:
#set up the depth we want to achieve, since it's a decision "tree"
CarTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)

# it shows the default parameters
CarTree 

In [ ]:
#finally fit the data into the model

CarTree.fit(decision_tree_df_x_trainset,decision_tree_df_y_trainset)

In [ ]:
# make the prediction
CarPredTree = CarTree.predict(decision_tree_df_x_testset)

In [ ]:
# compared with prediction and actual datasets to understand how accurate the model is
# formula of "accuracy" : # of correctly predict datasets in testset / total # datasets in testset 
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(decision_tree_df_y_testset, CarPredTree))

# KNN

In [ ]:
vehicle = vehicle.fillna(0)
KNN_df_x = vehicle[['Year','Odometer','Trim','Wreck','Cab','displ','CombMPG']].values
KNN_df_y = vehicle["Price"]

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
KNN_df_x = preprocessing.StandardScaler().fit(KNN_df_x).transform(KNN_df_x.astype(float))
KNN_df_x[0:5]
from sklearn.model_selection import train_test_split
KNN_df_x_train, KNN_df_x_test, KNN_df_y_train, KNN_df_y_test = train_test_split( KNN_df_x, KNN_df_y, test_size=0.2, random_state=4)
print ('Train set:', KNN_df_x_train.shape,  KNN_df_y_train.shape)
print ('Test set:', KNN_df_x_test.shape,  KNN_df_y_test.shape)

In [ ]:
Ks = 40
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(KNN_df_x_train,KNN_df_y_train)
    yhat=neigh.predict(KNN_df_x_test)
    mean_acc[n-1] = metrics.accuracy_score(KNN_df_y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==KNN_df_y_test)/np.sqrt(yhat.shape[0])

mean_acc

In [ ]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()